In [ ]:
from nbformat import reads, NO_CONVERT
from nbconvert import PythonExporter
def code_extractor(jpt):
    nb = reads(jpt, NO_CONVERT)
    exporter = PythonExporter()
    source, meta = exporter.from_notebook_node(nb)
    return source

In [ ]:
import imp
Matchmaker = imp.load_source('name', '../matchmaker.py')
import ast
def function_matching(nb):
    tree = ast.parse(nb)
    mm = Matchmaker.Matchmaker()
    mm.visit(tree)
    matchs = mm.matching()
    return matchs

# Matching the functions 1st version

In [ ]:
from pydriller import RepositoryMining
import difflib
import pandas as pd

def repo_matching(repo, commit, path):
    for c in RepositoryMining(repo, single = commit).traverse_commits():
        for modified_files in c.modifications:
            if modified_files.new_path == path:
                try:
                    before = code_extractor(modified_files.source_code_before)
                except TypeError:
                    before = ""
                lines_b = before.strip().splitlines()
                new = code_extractor(modified_files.source_code)
                lines_n = new.strip().splitlines()
                
                diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                lines = list(diff)[2:]
                lineno = 0
                changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
                for line in lines:
                    prefix = '@@'
                    if line.startswith(prefix):
                        s = line[line.find("+"):]
                        try:
                            lineno = int(s[1:s.find(",")])
                        except:
                            lineno = int(s[1:s.find(" ")])
                    else:
                        if line.startswith("+"):
                            changes = changes.append({"repo": c.project_name, 
                                                      "path": modified_files.new_path,
                                                      "author": c.author.email,
                                                      "commit": c.hash,
                                                      "line": lineno,
                                                      "code": line[1:]}, ignore_index= True)
                            lineno += 1
                matches = function_matching(new)
                result = pd.merge(matches, changes, how = 'inner', on= "line")
                return result

# Test

In [ ]:
from pydriller import RepositoryMining, GitRepository
repo = "/home/ubuntu/repos/abulbasar/machine-learning"
test = RepositoryMining(repo)
for commit in test.traverse_commits():
    for file in commit.modifications:
        print(commit.hash + ' at ' + file.new_path)

In [ ]:
repo = "/home/ubuntu/repos/abulbasar/machine-learning"
path = "01 Neural Network using Numpy.ipynb"
commit = "df41ae028af5a445ec41a94925ac98ad2ccdea01"

m = repo_matching(repo = repo,path = path,commit = commit)


In [ ]:
m.head()

# Running through all commits 1st version

In [ ]:
import pandas as pd
import ast
import csv
# incresing the csv field size
import ctypes
csv.field_size_limit(int(ctypes.c_ulong(-1).value // 2))
failed = []
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
repo_path = "/home/ubuntu/repos/"
with open('../data/commits_bigquery.csv',"r", encoding="utf8") as csvfile:
    with open("../data/repo_function_matched.csv", "w", encoding="utf8") as newfile:
        data = csv.DictReader(csvfile)
        writer = csv.DictWriter(newfile, fieldnames=fields)
        writer.writeheader()
        for row in data:
            try:
                commit = row['commit']
                repo = row['repo']
                path = row['path']
                #path = path.replace("/", "\\")
                repo = repo_path + repo
                matched = repo_matching(repo, commit, path)
                matched.to_csv(newfile, header=False, index= False)
            except KeyboardInterrupt:
                break
            except Exception as e: 
                failed.append(e)
                #print(e)
                #break
import pickle

with open('../data/fails', 'wb') as fp:
    pickle.dump(failed, fp)

In [ ]:
failed

# Prepare data 2nd version

In [ ]:
import pandas as pd
import numpy as np
import os.path
from os import path

df = pd.read_csv('../data/commits_bigquery.csv')
df = df.drop(df.columns[0], axis = 1)

repos = df.repo.unique()
add_path = np.vectorize(lambda x: "/home/ubuntu/repos/" +x)
repos = add_path(repos)
repos = repos.tolist()
for i in repos:
    if not path.exists(i):
        repos.remove(i)

commits = df.commit.unique().tolist()

paths = df[['commit','path']].groupby('commit')['path'].apply(list)

In [ ]:
len(repos)

# 2nd version

In [ ]:
from pydriller import RepositoryMining
import difflib
import pandas as pd
import csv
#inspect all repos and commits
failed = []
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
newfile = open("../data/repo_function_matched.csv", "w", encoding="utf8")
writer = csv.DictWriter(newfile, fieldnames=fields)
writer.writeheader()
    
for c in RepositoryMining(repos, only_no_merge=True, only_commits=commits).traverse_commits():
    for modified_files in c.modifications:
        try:
            if modified_files.new_path in paths[c.hash]:
                try:
                    before = code_extractor(modified_files.source_code_before)
                except TypeError:
                    before = ""
                lines_b = before.strip().splitlines()
                new = code_extractor(modified_files.source_code)
                lines_n = new.strip().splitlines()
                
                diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                lines = list(diff)[2:]
                lineno = 0
                changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
                for line in lines:
                    prefix = '@@'
                    if line.startswith(prefix):
                        s = line[line.find("+"):]
                        try:
                            lineno = int(s[1:s.find(",")])
                        except:
                            lineno = int(s[1:s.find(" ")])
                    else:
                        if line.startswith("+"):
                            changes = changes.append({"repo": c.project_name, 
                                                      "path": modified_files.new_path,
                                                      "author": c.author.email,
                                                      "commit": c.hash,
                                                      "line": lineno,
                                                      "code": line[1:]}, ignore_index= True)
                            lineno += 1
                matches = function_matching(new)
                result = pd.merge(matches, changes, how = 'inner', on= "line")
                result.to_csv(newfile, header=False, index= False)
        except KeyboardInterrupt:
            break
        except Exception as e: 
            failed.append([c.project_name,modified_files.new_path,e])

newfile.close()
                
import pickle

with open('../data/fails', 'wb') as fp:
    pickle.dump(failed, fp)

# Preparing data 3rd version

In [ ]:
import pandas as pd
import numpy as np
import os.path
from os import path

df = pd.read_csv('../data/commits_bigquery.csv')
df = df.drop(df.columns[0], axis = 1)

repos = df.repo.unique()
folder = "/home/ubuntu/repos/"
repos = repos.tolist()
for i in repos:
    if not path.exists(folder+i):
        repos.remove(i)
repos = repos.tolist()
folder = "/home/ubuntu/repos/"
commits = df[['repo','commit']].groupby('repo')['commit'].apply(list)
paths = df[['commit','path']].groupby('commit')['path'].apply(list)

# Running 3rd version

In [ ]:
from pydriller import RepositoryMining
import difflib
import pandas as pd
import csv
#inspect all repos and commits
failed = []
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
newfile = open("../data/repo_function_matched.csv", "w", encoding="utf8")
writer = csv.DictWriter(newfile, fieldnames=fields)
writer.writeheader()
    
for r in repos:
    try:
        final_path = folder+r
        repository = RepositoryMining(final_path, only_no_merge=True, only_commits=commits[r])
        for c in repository.traverse_commits():
            for f in c.modifications:
                try:
                    if f.new_path in paths[c.hash]:
                        try:
                            before = code_extractor(f.source_code_before)
                        except TypeError:
                            before = ""
                        lines_b = before.strip().splitlines()
                        new = code_extractor(f.source_code)
                        lines_n = new.strip().splitlines()
                        
                        diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                        lines = list(diff)[2:]
                        lineno = 0
                        changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
                        for line in lines:
                            prefix = '@@'
                            if line.startswith(prefix):
                                s = line[line.find("+"):]
                                try:
                                    lineno = int(s[1:s.find(",")])
                                except:
                                    lineno = int(s[1:s.find(" ")])
                            else:
                                if line.startswith("+"):
                                    changes = changes.append({"repo": c.project_name, 
                                                              "path": f.new_path,
                                                              "author": c.author.email,
                                                              "commit": c.hash,
                                                              "line": lineno,
                                                              "code": line[1:]}, ignore_index= True)
                                    lineno += 1
                        matches = function_matching(new)
                        result = pd.merge(matches, changes, how = 'inner', on= "line")
                        result.to_csv(newfile, header=False, index= False)
                        del result
                        del changes
                        del matches
                except KeyboardInterrupt:
                    break
                except Exception as e: 
                    failed.append(['File',f.new_path,e])
        del repository
    except KeyboardInterrupt:
        break
    except Exception as e: 
        failed.append(['Project',r,e])

newfile.close()
                
import pickle

with open('../data/fails', 'wb') as fp:
    pickle.dump(failed, fp)

# Running 4th version

In [ ]:
def file_matcher(c,f,newfile):
    if f.new_path in paths[c.hash]:
        try:
            before = code_extractor(f.source_code_before)
        except TypeError:
            before = ""
        lines_b = before.strip().splitlines()
        new = code_extractor(f.source_code)
        lines_n = new.strip().splitlines()
        
        diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
        lines = list(diff)[2:]
        lineno = 0
        changes = pd.DataFrame(columns = ['repo','path','author','commit','line','code'])
        for line in lines:
            prefix = '@@'
            if line.startswith(prefix):
                s = line[line.find("+"):]
                try:
                    lineno = int(s[1:s.find(",")])
                except:
                    lineno = int(s[1:s.find(" ")])
            else:
                if line.startswith("+"):
                    changes = changes.append({"repo": c.project_name, 
                                              "path": f.new_path,
                                              "author": c.author.email,
                                              "commit": c.hash,
                                              "line": lineno,
                                              "code": line[1:]}, ignore_index= True)
                    lineno += 1
        matches = function_matching(new)
        result = pd.merge(matches, changes, how = 'inner', on= "line")
        result.to_csv(newfile, mode='a', header=False, index= False)

In [ ]:
def repo_matcher(repo, commits, paths, newfile):
    final_path = folder+repo
    repository = RepositoryMining(final_path, only_no_merge=True, only_commits=commits[r])
    for c in repository.traverse_commits():
        for f in c.modifications:
            try:
                file_matcher(c,f,newfile)
            except KeyboardInterrupt:
                break
            except: 
                continue

In [ ]:
import pandas as pd
import numpy as np
import os.path
from os import path

df = pd.read_csv('../data/commits_bigquery.csv')
df = df.drop(df.columns[0], axis = 1)

repos = df.repo.unique()
folder = "/home/ubuntu/repos/"
repos = repos.tolist()
for i in repos:
    if not path.exists(folder+i):
        repos.remove(i)

commits = df[['repo','commit']].groupby('repo')['commit'].apply(list)
paths = df[['commit','path']].groupby('commit')['path'].apply(list)

In [ ]:
from pydriller import RepositoryMining
import difflib
import pandas as pd
import csv
#inspect all repos and commits
fields = ['line','function','package','class','repo','path', 'author', 'commit','code']
to_disk = "../data/repo_function_matched.csv"
with open(to_disk, "w", encoding="utf8") as newfile:
    writer = csv.DictWriter(newfile, fieldnames=fields)
    writer.writeheader()
    
for r in repos:
    try:
        repo_matcher(r, commits, paths, to_disk)
    except KeyboardInterrupt:
        break
    except: 
        continue

## 5th Version

In [ ]:
from nbformat import reads, NO_CONVERT
from nbconvert import PythonExporter
def code_extractor(jpt):
    nb = reads(jpt, NO_CONVERT)
    exporter = PythonExporter()
    source, meta = exporter.from_notebook_node(nb)
    return stripComments(source)

In [ ]:
import re
def stripComments(code):
    code = str(code)
    code = re.sub(r'^$\n', '', code, flags=re.MULTILINE)
    return re.sub(r'(?m)^ *#.*\n?', '', code)

In [ ]:
import imp
Matchmaker = imp.load_source('name', '../matchmaker.py')
import ast
def function_matching(nb):
    tree = ast.parse(nb)
    mm = Matchmaker.Matchmaker()
    mm.visit(tree)
    matchs = mm.matching()
    return matchs

In [ ]:
def file_matcher(c,f,newfile):
    if f.new_path in paths[c.hash]:
        try:
            before = code_extractor(f.source_code_before)
        except TypeError:
            before = ""
        lines_b = before.strip().splitlines()
        new = code_extractor(f.source_code)
        lines_n = new.strip().splitlines()
        
        columns = ['repo','path','author','commit','line','code']
        temp_path = "../data/temp.csv"
        with open(temp_path, "w", encoding="utf8") as temp:
            writer = csv.DictWriter(temp, fieldnames=fields)
            writer.writeheader()
            if before != "":
                diff = difflib.unified_diff(lines_b, lines_n, fromfile='before', tofile='new', lineterm='', n=0)
                lines = list(diff)[2:]
                lineno = 0
                for line in lines:
                    prefix = '@@'
                    if line.startswith(prefix):
                        s = line[line.find("+"):]
                        try:
                            lineno = int(s[1:s.find(",")])
                        except:
                            lineno = int(s[1:s.find(" ")])
                    else:
                        if line.startswith("+"):
                            writer.writerow({"repo": c.project_name, 
                                             "path": f.new_path,
                                             "author": c.author.email,
                                             "commit": c.hash,
                                             "line": lineno,
                                             "code": line[1:]})
                            lineno += 1
            else:
                for idx,line in enumerate(lines_n):
                    writer.writerow({"repo": c.project_name, 
                                     "path": f.new_path,
                                     "author": c.author.email,
                                     "commit": c.hash,
                                     "line": idx+1,
                                     "code": line})
            
                
        changes = pd.read_csv(temp_path)
        changes['line'] = changes['line'].astype(int)
        matches = function_matching(new)
        matches['line'] = matches['line'].astype(int)
        result = pd.merge(matches, changes, how = 'inner', on= "line")
        result.to_csv(newfile, mode='a', header=False, index= False)

In [ ]:
def repo_matcher(repo, commits, paths, newfile):
    final_path = folder+repo
    repository = RepositoryMining(final_path, only_no_merge=True, only_commits=commits[r])
    for c in repository.traverse_commits():
        for f in c.modifications:
            try:
                file_matcher(c,f,newfile)
            except KeyboardInterrupt:
                break
            except: 
                continue

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import os.path
from os import path

df = pd.read_csv('../data/commits_bigquery.csv')
df = df.drop(df.columns[0], axis = 1)
data = pd.read_csv('../data/repo_function_matched_v4.csv')
un = data[['path','commit']].drop_duplicates()

df_all = df.merge(un, on=['path','commit'], how='left', indicator=True)
df_all = df_all[df_all['_merge'] == 'left_only']

repos = df_all.repo.unique()
folder = "/home/ubuntu/repos/"
repos = repos.tolist()
for i in repos:
    if not path.exists(folder+i):
        repos.remove(i)


commits = df_all[['repo','commit']].groupby('repo')['commit'].apply(list)
paths = df_all[['commit','path']].groupby('commit')['path'].apply(list)

In [ ]:
from pydriller import RepositoryMining
import difflib
import pandas as pd
import csv
#inspect all repos and commits
to_disk = "../data/repo_function_matched.csv"
    
for r in repos:
    try:
        repo_matcher(r, commits, paths, to_disk)
    except KeyboardInterrupt:
        break
    except: 
        continue